## Show allen atlas and probe trajectories

In [ ]:
import pyvista as pv
from vvasp import NeuropixelsChronicHolder, CustomMeshObject, Probe, VVASPAtlas

### Define the insertion data
You can define insertions (1) via angles, an AP/ML entrypoint, and a depth along the shank (from the pia), or (2) with the AP/ML/DV tip position and angles. This example shows the first method.

In [ ]:
insertion_data = [dict(entry=[-1578.0, -2485.0], # AP, ML
                       depth=3110,
                       angles=[115, 0, -10]),
                  dict(entry=[-76, 1730],
                       depth=4510,
                       angles=[135, 0, 110]),
                  dict(entry=[-2221, 1313],
                       depth=3801,
                       angles=[85, 0, 70]),]

regions_to_show = ['VISp','CP','ACA','MOp']

### Initialize the plotter and add the atlas and probes to the scene

In [ ]:
# initialize plotter
plotter = pv.Plotter(notebook=True) # blocking plotter in the notebook
#from pyvistaqt import BackgroundPlotter
#plotter = BackgroundPlotter() # non blocking plotter in a separate QT window

# add the atlas to scene and show regions (defaults to Allen CCF atlas)
atlas = VVASPAtlas(plotter, min_tree_depth=6, max_tree_depth=8, mapping=None)

for region in regions_to_show:
    atlas.add_atlas_region_mesh(region)

# add probes to scene
prbs = []
for ins in insertion_data:

    prb = NeuropixelsChronicHolder('NP24','head_fixed', plotter) # This object renders shanks AND the indie chronic holder 
    #prb = Probe('NP24',plotter) # this object renders only the shanks

    prb.drive_probe_from_entry(ins['entry'], ins['angles'], ins['depth'], root_mesh=atlas.meshes['root']) # position the probe via an entry point, angles and driven depth
    #prb.set_location(ins['tip'],ins['angles']) # use this method to position probes if you know the probe tip location instead of the entry point

    prb.make_inactive() # color shanks black
    prbs.append(prb)
plotter.isometric_view()

In [ ]:
# now show the scene
plotter.show() 
#plotter.show(jupyter_backend='client') # 'client' looks and performs better than 'server' on my machine, but doesn't allow further programmatic updating of the scene

In [ ]:
# show the regions that the shanks for one probe pass through
_, acronyms = prbs[0].compute_region_intersections(atlas)
for i,a in enumerate(acronyms):
    print(f'Shank {i}: {a}')

### Programatically set camera position - see PyVista docs for more info

In [ ]:
plotter.camera_position = [
    (0,0,60_000),
    (0,0,0),
    (0,0,0),
]
plotter.update()

### Real time updates of the pyvista scene

In [ ]:
# update the probe positions, the plotter will update in real time
for p in prbs:
    p.move('left',1000)
plotter.update() # force the plotter to redraw the scene

### Render your own custom meshes to the scene

In [ ]:
# add a custom mesh to the scene
import os
from pathlib import Path
meshpath = Path(os.path.abspath("")).parent / 'meshes' / 'logo.stl'

# you will need to supply some info about your mesh so that the mesh is properly oriented in the scene
scale_factor = 1000 # most meshes are in mm, but the pyvista scene is in um, so you will often need to scale your mesh
mesh_rotation = [0,0,0] # defines the x,y,z rotations that should be applied to the mesh when it's first loaded, in case the coordinate system of the mesh is different from the coordinate system of the scene
mesh_origin = [0,0,0] # this should be the origin that the mesh is moved and rotated about if you call .set_location() .move() or .rotate()
obj = CustomMeshObject(meshpath,
                       plotter, 
                       scale_factor=scale_factor, 
                       mesh_origin=mesh_origin, 
                       mesh_rotation=mesh_rotation,
                       pyvista_mesh_args=dict(color='red', opacity=0.5))

In [ ]:
obj.move('left',1000)
plotter.update()